## Install twint, optimus and reqs

In [9]:
!pip install --user -r requirements.txt

  Using cached optimuspyspark-2.2.29-py3-none-any.whl (39.1 MB)
  Using cached aiohttp-3.6.2-cp37-cp37m-manylinux1_x86_64.whl (1.2 MB)
  Using cached aiodns-2.0.0-py2.py3-none-any.whl (4.8 kB)
  Using cached beautifulsoup4-4.9.1-py3-none-any.whl (115 kB)
  Using cached cchardet-2.1.6-cp37-cp37m-manylinux2010_x86_64.whl (242 kB)
  Using cached elasticsearch-7.8.0-py2.py3-none-any.whl (188 kB)
  Using cached aiohttp_socks-0.3.9-py3-none-any.whl (17 kB)
  Using cached schedule-0.6.0-py2.py3-none-any.whl (8.7 kB)
  Using cached geopy-2.0.0-py3-none-any.whl (111 kB)
  Using cached nest_asyncio-1.3.3-py3-none-any.whl (4.7 kB)
  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached wordcloud-1.7.0-cp37-cp37m-manylinux1_x86_64.whl (364 kB)
  Using cached deepdiff-4.0.6-py3-none-any.whl (47 kB)
  Using cached fastnumbers-2.2.1-cp37-cp37m-manylinux1_x86_64.whl (55 kB)
  Using cached psutil-5.6.3.tar.gz (435 kB)
Processing /home/jovyan/.cache/pip/wheels/7b/6e/09/d25c862eee7518

  Using cached prompt_toolkit-2.0.10-py3-none-any.whl (340 kB)
Processing /home/jovyan/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6/wrapt-1.12.1-py3-none-any.whl
  Using cached joblib-0.16.0-py3-none-any.whl (300 kB)
  Using cached regex-2020.6.8-cp37-cp37m-manylinux2010_x86_64.whl (661 kB)


  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python3.7 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-5dxco19i/psutil/setup.py'"'"'; __file__='"'"'/tmp/pip-install-5dxco19i/psutil/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-d9zmqllm
       cwd: /tmp/pip-install-5dxco19i/psutil/
  Complete output (41 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/psutil
  copying psutil/_compat.py -> build/lib.linux-x86_64-3.7/psutil
  copying psutil/_common.py -> build/lib.linux-x86_64-3.7/psutil
  copying psutil/__init__.py -> build/lib.linux-x86_64-3.7/psutil
  copying psutil/_psosx.py -> build/lib.linux-x86_64-3.7/psutil
  copying psutil/_pswindows.py -> build/lib.linux-x8

In [22]:
!pip install git+https://github.com/twintproject/twint.git@origin/master#egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-9cfiemi9/twint
  Running command git clone -q https://github.com/twintproject/twint.git /tmp/pip-install-9cfiemi9/twint
  ERROR: Error [Errno 2] No such file or directory: 'git': 'git' while executing command git clone -q https://github.com/twintproject/twint.git /tmp/pip-install-9cfiemi9/twint
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [15]:
!mv src/twint .
!rm -r src

mv: cannot stat 'src/twint': No such file or directory
rm: cannot remove 'src': No such file or directory


In [16]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("twint/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import twint

ModuleNotFoundError: No module named 'twint'

In [ ]:
from optimus import Optimus
op = Optimus()

## Setup twint

In [ ]:
# Set up TWINT config
c = twint.Config()

In [ ]:
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio
nest_asyncio.apply()

# Search for data science tweets

In [ ]:
c.Search = "data science"
# Custom output format
c.Format = "Username: {username} |  Tweet: {tweet}"
c.Limit = 1
c.Pandas = True

twint.run.Search(c)

## Saving results into Pandas

In [ ]:
def available_columns():
    return twint.output.panda.Tweets_df.columns

In [ ]:
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

In [ ]:
available_columns()

In [ ]:
df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])

In [ ]:
df_pd

# Sentiment analysis

## Save data to Optimus

In [ ]:
df = op.create.data_frame(pdf= df_pd)

### Clean tweets

In [ ]:
clean_tweets = df.cols.remove_accents("tweet") \
                 .cols.remove_special_chars("tweet")

In [ ]:
clean_tweets.count()

In [ ]:
tweets = clean_tweets.select("tweet").rdd.flatMap(lambda x: x).collect()

In [ ]:
tweets

### Analyze sentiment of tweets

In [ ]:
from textblob import TextBlob
from IPython.display import Markdown, display

In [ ]:
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [ ]:
for tweet in tweets:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("Neutral", color="grey")
        print("")

## Add sentiments to dataframe

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative

In [ ]:
sentiment = udf(apply_blob)

In [ ]:
clean_tweets.withColumn("sentiment", sentiment(clean_tweets['tweet'])).show()

# Making the code modular

## Setup everything

In [ ]:
%load_ext autoreload
%autoreload 2

# Import twint
import sys
sys.path.append("twint/")

# Set up TWINT config
import twint
c = twint.Config()

# Other imports
import seaborn as sns
import os
from optimus import Optimus
op = Optimus()

# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio
nest_asyncio.apply()

# Disable annoying printing

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## The code with the magic

In [ ]:
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Function to get sentiment 
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative

# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())

# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

def tweets_sentiment(search, limit=1):
    c.Search = search
    # Custom output format
    c.Format = "Username: {username} |  Tweet: {tweet}"
    c.Limit = limit
    c.Pandas = True
    with HiddenPrints():
        print(twint.run.Search(c))
    
    # Transform tweets to pandas DF
    df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
    
    # Transform Pandas DF to Optimus/Spark DF
    df = op.create.data_frame(pdf= df_pd)
    
    # Clean tweets
    clean_tweets = df.cols.remove_accents("tweet") \
                 .cols.remove_special_chars("tweet")
    
    # Add sentiment to final DF
    return clean_tweets.withColumn("sentiment", sentiment(clean_tweets['tweet']))

In [ ]:
df_result = tweets_sentiment("data science", limit=1)

In [ ]:
df_result.show()

In [ ]:
df_result.count()

In [ ]:
df_result.printSchema()

### Lets see the distribution of the sentiments

In [ ]:
df_res_pandas = df_result.toPandas()

In [ ]:
sns.distplot(df_res_pandas['sentiment'])
sns.set(rc={'figure.figsize':(11.7,8.27)})

# Lets do more with Twint

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append("twint/")

import twint
import numpy as np
from PIL import Image
from os import path

# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Disable annoying printing

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## Get tweets easily

In [ ]:
def get_tweets(search, limit=100):
    c = twint.Config()
    c.Search = search
    c.Limit = limit
    c.Pandas = True
    c.Pandas_clean = True

    with HiddenPrints():
        print(twint.run.Search(c))
    return twint.output.panda.Tweets_df[["username","tweet"]]

In [ ]:
tweets = get_tweets("data science", limit=10000)

In [ ]:
tweets.count()

## Generate word cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline

text = tweets.tweet.values

# adding movie script specific stopwords
stopwords = set(STOPWORDS)
stopwords.add("https")
stopwords.add("xa0")
stopwords.add("xa0'")
stopwords.add("bitly")
stopwords.add("bit")
stopwords.add("ly")
stopwords.add("twitter")
stopwords.add("pic")

wordcloud = WordCloud(
    background_color = 'black',
    width = 1000,
    height = 500,
    stopwords = stopwords).generate(str(text))

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
def generate_word_cloud(tweets):
    
    # Getting the text out of the tweets
    text = tweets.tweet.values
    
    # adding movie script specific stopwords
    stopwords = set(STOPWORDS)
    stopwords.add("https")
    stopwords.add("xa0")
    stopwords.add("xa0'")
    stopwords.add("bitly")
    stopwords.add("bit")
    stopwords.add("ly")
    stopwords.add("twitter")
    stopwords.add("pic")

    wordcloud = WordCloud(
        background_color = 'black',
        width = 1000,
        height = 500,
        stopwords = stopwords).generate(str(text))
    
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.rcParams['figure.figsize'] = [10, 10]

## Other example

In [ ]:
tweets = get_tweets("artificial intelligence", limit=1000)

In [ ]:
generate_word_cloud(tweets)

## Scaling up

In [ ]:
df_result = tweets_sentiment("data science", limit=10000)
df_result.show()

In [ ]:
df_result.count()